# Speech T5-VC / Microsoft

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets
!pip install sentencepiece

# Text-to-Speech

In [2]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
inputs = processor(text="What do you do today and where are you?", return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 4, 38, 11,  7,  6,  4, 14,  8,  4, 22,  8, 16,  4, 14,  8,  4,  6,  8,
         14,  7, 22,  4,  7,  9, 14,  4, 20, 11,  5, 13,  5,  4,  7, 13,  5,  4,
         22,  8, 16, 41,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [4]:
from datasets import load_dataset
import torch

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7500]["xvector"]).unsqueeze(0)

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset arctic_xvectors downloaded and prepared to /root/.cache/huggingface/datasets/Matthijs___arctic_xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f. Subsequent calls will reuse this data.


In [5]:
speaker_embeddings.shape

torch.Size([1, 512])

In [6]:
spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)
print(spectrogram)

tensor([[-4.1236, -4.2379, -4.3097,  ..., -4.7585, -4.7624, -4.9068],
        [-3.8556, -3.9310, -3.9728,  ..., -4.6329, -4.6131, -4.8286],
        [-3.3025, -3.3714, -3.3123,  ..., -4.3762, -4.4063, -4.5948],
        ...,
        [-3.3556, -3.4055, -3.5259,  ..., -4.5421, -4.5751, -4.7718],
        [-3.6209, -3.7169, -3.8158,  ..., -4.6107, -4.6272, -4.8073],
        [-3.7195, -3.8373, -3.9315,  ..., -4.6042, -4.6184, -4.7966]])


In [7]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

In [8]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
print(speech.shape)

from IPython.display import Audio
Audio(speech, rate=16000)

torch.Size([31744])


In [9]:
import soundfile as sf

sf.write("tts_ex.wav", speech.numpy(), samplerate=16000)

# Speech-to-Speech

In [10]:
from transformers import SpeechT5ForSpeechToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_vc")
model = SpeechT5ForSpeechToSpeech.from_pretrained("microsoft/speecht5_vc")

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/621M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/speecht5_vc were not used when initializing SpeechT5ForSpeechToSpeech: ['speecht5.encoder.prenet.pos_conv_embed.conv.weight_g', 'speecht5.encoder.prenet.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing SpeechT5ForSpeechToSpeech from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SpeechT5ForSpeechToSpeech from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SpeechT5ForSpeechToSpeech were not initialized from the model checkpoint at microsoft/speecht5_vc and are newly initialized: ['speecht5.encoder.prenet.pos_conv_embed.conv.parametrizations.weight.original0', 'speecht5.encoder.prenet.pos_conv_embed.conv.parametrizati

In [25]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
len(dataset['audio'])

73

In [24]:
example = dataset[72]
Audio(example['audio']['array'], rate=16000)

In [36]:
sampling_rate = dataset.features['audio'].sampling_rate
inputs = processor(audio=example['audio']['array'], sampling_rate=sampling_rate, return_tensors='pt')

In [37]:
speaker_embeddings = torch.tensor(embeddings_dataset[7306]['xvector']).unsqueeze(0)

In [38]:
speech = model.generate_speech(inputs['input_values'], speaker_embeddings, vocoder=vocoder)

In [39]:
Audio(speech, rate=16000)

In [40]:
import soundfile as sf
sf.write("speech_converted.wav", speech.numpy(), samplerate=16000)

# Automatic Speech Recognition (Using Pipeline)

In [41]:
from transformers import pipeline

generator = pipeline(task="automatic-speech-recognition", model="microsoft/speecht5_asr")

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/606M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/speecht5_asr were not used when initializing SpeechT5ForSpeechToText: ['speecht5.encoder.prenet.pos_conv_embed.conv.weight_g', 'speecht5.encoder.prenet.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing SpeechT5ForSpeechToText from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SpeechT5ForSpeechToText from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SpeechT5ForSpeechToText were not initialized from the model checkpoint at microsoft/speecht5_asr and are newly initialized: ['speecht5.encoder.prenet.pos_conv_embed.conv.parametrizations.weight.original0', 'speecht5.encoder.prenet.pos_conv_embed.conv.parametrizations.we

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

In [42]:
transcription = generator(example['audio']['array'])
transcription

{'text': 'then the powerful twist that thrust it aside in and under the guard'}

In [43]:
from transformers import SpeechT5ForSpeechToText

processor = SpeechT5Processor.from_pretrained('microsoft/speecht5_asr')
model = SpeechT5ForSpeechToText.from_pretrained('microsoft/speecht5_asr')

Some weights of the model checkpoint at microsoft/speecht5_asr were not used when initializing SpeechT5ForSpeechToText: ['speecht5.encoder.prenet.pos_conv_embed.conv.weight_g', 'speecht5.encoder.prenet.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing SpeechT5ForSpeechToText from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SpeechT5ForSpeechToText from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SpeechT5ForSpeechToText were not initialized from the model checkpoint at microsoft/speecht5_asr and are newly initialized: ['speecht5.encoder.prenet.pos_conv_embed.conv.parametrizations.weight.original0', 'speecht5.encoder.prenet.pos_conv_embed.conv.parametrizations.we

In [44]:
sampling_rate = dataset.features['audio'].sampling_rate
inputs = processor(audio=example['audio']['array'], sampling_rate=sampling_rate, return_tensors='pt')

In [45]:
predicted_ids = model.generate(**inputs, max_length=100)
predicted_ids

tensor([[ 2,  4,  6, 11,  5,  9,  4,  6, 11,  5,  4, 24,  8, 20,  5, 13, 19, 16,
         15,  4,  6, 20, 10, 12,  6,  4,  6, 11,  7,  6,  4,  6, 11, 13, 16, 12,
          6,  4, 10,  6,  4,  7, 12, 10, 14,  5,  4, 10,  9,  4,  7,  9, 14,  4,
         16,  9, 14,  5, 13,  4,  6, 11,  5,  4, 21, 16,  7, 13, 14,  2]])

In [47]:
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
transcription

['then the powerful twist that thrust it aside in and under the guard']